# Lab: Sinusoid Parameter Estimation in ECG Using Least Squares

This notebook is part of the **Statistical Signal Processing** course.

**Goal:** Apply the Least Squares Estimator (LSE) to estimate the amplitude and phase of a low-frequency sinusoidal component embedded in an ECG signal.

You will:
- Download a real ECG record from PhysioNet.
- Select a short time interval.
- Identify a low-frequency component using the spectrum.
- Build a linear sinusoidal model and estimate its parameters using LSE.
- Evaluate the quality of the fit and interpret the results.


## 1. Dataset and Setup

We will use the **MIT-BIH Normal Sinus Rhythm Database (nsrdb)** from PhysioNet.

- PhysioNet page: https://physionet.org/content/nsrdb/1.0.0/
- Choose a record, for example: `16265`.

### Instructions
1. Run the cell below to install and import required Python packages.
2. Then, use WFDB to download and read the ECG record from PhysioNet.


In [ ]:
# 1. Install and import required packages

!pip install wfdb plotly

import wfdb
import numpy as np
import plotly.graph_objects as go


In [ ]:
# 2. Download and read an ECG record from PhysioNet (nsrdb)

record_name = '16272'   # you may change the record
database = 'nsrdb'

record = wfdb.rdrecord(record_name, pn_dir=database)

ecg = record.p_signal[:, 0]   # first ECG channel
fs = record.fs                # sampling frequency

print(f"Sampling frequency: {fs} Hz")
print(f"Total number of samples: {len(ecg)}")


In [ ]:
# 3. Select a short segment (e.g., 20 seconds) and plot it (interactive)

duration_sec = 20
start_sec = 0

start_sample = int(start_sec * fs)
end_sample = int((start_sec + duration_sec) * fs)

ecg_seg = ecg[start_sample:end_sample]
N = len(ecg_seg)
t = np.arange(N) / fs

print(f"Segment length: {N} samples ({duration_sec} seconds)")

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=ecg_seg, mode="lines", name="ECG segment"))
fig.update_layout(
    title=f"ECG record {record_name} - segment from {start_sec}s to {start_sec + duration_sec}s",
    xaxis_title="Time [s]",
    yaxis_title="Amplitude [mV]",
)
fig.show()


## 2. Frequency Analysis and Selection of the Sinusoidal Component

We assume that the ECG segment contains a **low-frequency oscillation** (e.g., due to respiration or baseline wander).

In this part you will:
1. Remove the DC component of the segment (mean value).
2. Compute the magnitude spectrum using the Discrete Fourier Transform (DFT).
3. Identify a low-frequency peak (typically in the range 0.1–0.3 Hz).
4. Choose a frequency $ f_0 $ and compute the corresponding angular frequency:

$$
\omega_0 = 2\pi \frac{f_0}{f_s}.
$$


In [ ]:
# 4. Remove DC and compute the magnitude spectrum of the ECG segment

ecg_seg_dc = ecg_seg - np.mean(ecg_seg)

X = np.fft.rfft(ecg_seg_dc)
freqs = np.fft.rfftfreq(N, d=1/fs)
X_mag = np.abs(X)

max_freq_to_show = 2.0
mask = freqs <= max_freq_to_show
freqs_low = freqs[mask]
X_mag_low = X_mag[mask]

fig = go.Figure()
fig.add_trace(go.Scatter(x=freqs_low, y=X_mag_low, mode="lines+markers", name="|X(f)|"))
fig.update_layout(
    title="Magnitude spectrum (low frequencies)",
    xaxis_title="Frequency [Hz]",
    yaxis_title="|X(f)|",
)
fig.show()


### Choose the sinusoidal frequency

From the spectrum above, visually select a **low-frequency peak** (for example, around 0.1–0.3 Hz) and set:

- The chosen frequency $ f_0 $ (in Hz).
- The corresponding angular frequency $ \omega_0 $:

$$
\omega_0 = 2\pi \frac{f_0}{f_s}.
$$


In [ ]:
# 5. Define f0 (in Hz) and compute omega0 (in rad/sample)

f0 = 0.2  # example value, adjust after inspecting the spectrum
omega0 = 2 * np.pi * f0 / fs
print(f"Chosen f0 = {f0} Hz")
print(f"omega0 = {omega0} rad/sample")


## 3. Construction of the LSE Model

We model the ECG segment as:

$$
x[n] \approx \alpha \cos(\omega_0 n) + \beta \sin(\omega_0 n) + w[n],
$$

which is **linear in the parameters** $ \alpha $ and $ \beta $.

We define:

$$
x = \begin{bmatrix} x[0] \\ x[1] \\ \vdots \\ x[N-1] \end{bmatrix},
\quad
X = \begin{bmatrix}
\cos(\omega_0 \cdot 0) & \sin(\omega_0 \cdot 0) \\
\cos(\omega_0 \cdot 1) & \sin(\omega_0 \cdot 1) \\
\vdots & \vdots \\
\cos(\omega_0 (N-1)) & \sin(\omega_0 (N-1))
\end{bmatrix}.
$$

The parameter vector is:

$$
\boldsymbol{\theta} = \begin{bmatrix} \alpha \\ \beta \end{bmatrix},
$$

and the **Least Squares Estimator (LSE)** is:

$$
\hat{\boldsymbol{\theta}} = (H^\top H)^{-1} H^\top x.
$$

From the estimated parameters we obtain the amplitude and phase:

$$
A = \sqrt{\hat{\alpha}^2 + \hat{\beta}^2},
\qquad
\phi = \arctan\!\left(-\frac{\hat{\beta}}{\hat{\alpha}}\right).
$$


In [ ]:
# 6. Build the design matrix X and solve the LSE for alpha and beta

x_vec = ecg_seg_dc
n = np.arange(N)
X = np.column_stack((np.cos(omega0 * n), np.sin(omega0 * n)))

XtX = X.T @ X
Xtx = X.T @ x_vec
theta_hat = np.linalg.inv(XtX) @ Xtx

alpha_hat = theta_hat[0]
beta_hat = theta_hat[1]

A_hat = np.sqrt(alpha_hat**2 + beta_hat**2)
phi_hat = np.arctan(-beta_hat / alpha_hat)

print(f"alpha_hat = {alpha_hat}")
print(f"beta_hat  = {beta_hat}")
print(f"Estimated amplitude A = {A_hat}")
print(f"Estimated phase phi = {phi_hat} rad")


## 4. Model Validation

We reconstruct the fitted sinusoid:

$$
\hat{x}[n] = \hat{\alpha}\cos(\omega_0 n) + \hat{\beta}\sin(\omega_0 n), \quad n = 0,1,\dots,N-1.
$$

Then we compute the **residual signal**:

$$
r[n] = x[n] - \hat{x}[n].
$$

We will evaluate the fit using:

- Mean Squared Error (MSE)
- Variance of the residuals
- Explained Variance (EV). It is a quantitative measure of how well a model captures the variability (energy) of the original signal. In this example, it would defined as describe in the following equation. Search on the internet and the documentation about this parameter: What is it? When is it useful? How is it calculated? What information does it provide?

$$
\text{EV} = 1 - \frac{\sum r[n]^2}{\sum (x[n] - \bar{x})^2}.
$$


In [ ]:
# 7. Reconstruct the fitted sinusoid and compute residuals (interactive plots)

x_hat = alpha_hat * np.cos(omega0 * n) + beta_hat * np.sin(omega0 * n)
r = x_vec - x_hat

MSE = np.mean(r**2)
r_var = np.var(r)
x_centered = x_vec - np.mean(x_vec)
EV = 1 - np.sum(r**2) / np.sum(x_centered**2)

print(f"MSE = {MSE}")
print(f"Residual variance = {r_var}")
print(f"Explained variance (EV) = {EV}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=x_vec, mode="lines", name="ECG segment (DC-removed)"))
fig.add_trace(go.Scatter(x=t, y=x_hat, mode="lines", name="Fitted sinusoid",
                          line=dict(dash="dash")))
fig.update_layout(
    title="ECG segment vs. fitted sinusoid",
    xaxis_title="Time [s]",
    yaxis_title="Amplitude",
)
fig.show()

fig_r = go.Figure()
fig_r.add_trace(go.Scatter(x=t, y=r, mode="lines", name="Residual"))
fig_r.update_layout(
    title="Residual signal r[n]",
    xaxis_title="Time [s]",
    yaxis_title="Residual",
)
fig_r.show()


## 5. Interpretation and Discussion

Answer the following questions:

1. Does the sinusoidal model capture a significant low-frequency component of the ECG segment?
2. Is the estimated amplitude $ A $ physically reasonable for baseline wander or respiratory modulation?
3. How large is the explained variance (EV)? Would you consider the model good, fair, or poor?
4. What are the main limitations of modeling an ECG segment using only one sinusoidal component?
5. How could this modeling approach be improved (e.g., multiple sinusoids, time-varying models, filtering)?


## 6. Extension to the ultrasonic transducer example.

For the example included in the thoery notebook, evaluate the fit calculatig:

- Mean Squared Error (MSE)
- Variance of the residuals
- Explained Variance (EV)

